In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import copy
import pdb
import pandas as pd

# pytorch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.datasets import ImageFolder
from torchvision import models, transforms

In [2]:
seed = 211
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomRotation(10),
        transforms.RandomHorizontalFlip(),
        transforms.RandomAffine(degrees=0, scale=(1.0, 1.1), shear=0),      # zoom
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [3]:
def set_parameter_requires_grad(model, feature_extracting=False):

  if feature_extracting:
      # frozen model
      model.requires_grad_(False)
  else:
      # fine-tuning
      model.requires_grad_(True)

In [4]:
# for LoRA/DoRA
class LowRankLayer(nn.Module):
    def __init__(self, linear, rank, alpha, use_dora=True):
        super().__init__()
        # rank: controls the inner dimension of the matrices A and B; controls the number of additional parameters introduced by LoRA,
        # a key factor in determining the balance between model adaptability and parameter efficiency.
        # alpha: a scaling hyper-parameter applied to the output of the low-rank adaptation,
        # controls the extent to which the adapted layer's output is allowed to influence the original output of the layer being adapted.

        self.use_dora = use_dora
        self.rank = rank  # low-rank
        self.alpha = alpha  # scaling hyper-parameter
        self.linear = linear
        self.in_dim = linear.in_features
        self.out_dim = linear.out_features

        # weights
        std_dev = 1 / torch.sqrt(torch.tensor(self.rank).float())
        self.A = nn.Parameter(torch.randn(self.in_dim, self.rank) * std_dev)
        self.B = nn.Parameter(torch.zeros(self.rank, self.out_dim))

        if self.use_dora:
            self.m = nn.Parameter(
                self.linear.weight.norm(p=2, dim=0, keepdim=True))
        else:
            self.m = None

    def forward(self, x):
        lora = self.A @ self.B  # combine LoRA matrices
        if self.use_dora:
            numerator = self.linear.weight + self.alpha * lora.T
            denominator = numerator.norm(p=2, dim=0, keepdim=True)
            directional_component = numerator / denominator
            new_weight = self.m * directional_component
            return F.linear(x, new_weight, self.linear.bias)
        else:
            # combine LoRA with orig. weights
            combined_weight = self.linear.weight + self.alpha * lora.T
            return F.linear(x, combined_weight, self.linear.bias)

In [15]:
def initialize_model(model_name = None, num_classes = 2, feature_extract = True, use_pretrained = True, DoRA = False, LoRA = False):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model = None
    weights = 'DEFAULT' if use_pretrained else None
    if model_name == "mobile net 2" or model_name == "MN_v2":
        model = models.mobilenet_v2(weights=weights)
        set_parameter_requires_grad(model, feature_extract)
        if DoRA or LoRA:
            rank, alpha = 4, 8
            use_dora = True if DoRA else False
            model.classifier[0] = LowRankLayer(model.classifier[0], rank, alpha, use_dora=use_dora)
        num_ftrs = model.classifier[1].in_features       # 1280
        model.classifier[1] = nn.Linear(num_ftrs, num_classes)  # replace the last FC layer
        input_size = 224
        
    elif model_name == "mobile net 3 large" or model_name == "MN_v3_l":
        model = models.mobilenet_v3_large(weights=weights)
        set_parameter_requires_grad(model, feature_extract)
        if DoRA or LoRA:
            rank, alpha = 4, 8
            use_dora = True if DoRA else False
            model.classifier[0] = LowRankLayer(model.classifier[0], rank, alpha, use_dora=use_dora)
        num_ftrs = model.classifier[3].in_features       # 1280
        model.classifier[3] = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "mobile net 3 small" or model_name == "MN_v3_s":
        model = models.mobilenet_v3_small(weights=weights)
        set_parameter_requires_grad(model, feature_extract)
        if DoRA or LoRA:
            rank, alpha = 4, 8
            use_dora = True if DoRA else False
            model.classifier[0] = LowRankLayer(model.classifier[0], rank, alpha, use_dora=use_dora)
        num_ftrs = model.classifier[3].in_features       # 1024
        model.classifier[3] = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "squeezenet" or model_name == "SQN": # just to show 1X1 convolutional replacement instead of fully connected
        model = models.squeezenet1_0(weights=weights)
        set_parameter_requires_grad(model, feature_extract)
        if DoRA or LoRA:
            rank, alpha = 4, 8
            use_dora = True if DoRA else False
            model.classifier[1] = LowRankLayer(model.classifier[1], rank, alpha, use_dora=use_dora)
        model.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1, 1), stride=(1, 1))
        model.num_classes = num_classes
        input_size = 224
        
    elif model_name == "resnet50" or model_name == "RN50":
        model = models.resnet50(weights=weights)
        set_parameter_requires_grad(model, feature_extract)        
        num_ftrs = model.fc.in_features
        if DoRA or LoRA:
            rank, alpha = 4, 8
            use_dora = True if DoRA else False
            model.fc = LowRankLayer(model.fc, rank, alpha, use_dora=use_dora)
        model.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "vit base 16" or model_name == "VIT":     # visual transformer model
        model = models.vit_b_16(weights=weights)
        set_parameter_requires_grad(model, feature_extract)
        num_ftrs = model.heads[0].in_features
        model.heads[0] = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    else:
        raise NotImplementedError

    return model, input_size 

In [6]:
data_dir = r'C:\Users\Data_Science\Documents\GitHub\deep-learn-project\input\dataset'   # change to your directory
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [7]:
def create_dataloaders(batch_size = 32, data_dir = None):
    image_datasets = {x: ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'test']}
    val_size = int(0.2 * len(image_datasets['train']))
    train_size = len(image_datasets['train']) - val_size
    train_set, val_set = random_split(image_datasets['train'], [train_size, val_size])
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True, num_workers=4)
    test_loader = DataLoader(image_datasets['test'], batch_size=batch_size, shuffle=True, num_workers=4)
    dataloaders = {
        'train': train_loader,
        'val': val_loader,
        'test': test_loader
    }
    class_names = image_datasets['train'].classes
    return dataloaders, class_names


In [8]:
def apply_optimizer(model, feature_extract):
    params_to_update = model.parameters()
    print("Params to learn:")
    if feature_extract:
        params_to_update = []  # override the initial list definition above
        for name, param in model.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                print("\t", name)
    else:
        for name, param in model.named_parameters():
            if param.requires_grad == True:
                print("\t", name)
    # Observe that all parameters are being optimized
    optimizer = torch.optim.SGD(params_to_update, lr=0.001, momentum=0.9)
    return optimizer

In [9]:
# training function
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()
    model = model.to(device)

    train_loss, train_acc, val_loss, val_acc = [], [], [], []
    history = [train_loss, train_acc, val_loss, val_acc]


    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        # zero the parameter gradients
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
            # epoch statistics to numpy
            # epoch_loss = epoch_loss.numpy()
            epoch_acc = epoch_acc.cpu().numpy()

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'train':
                train_loss.append(epoch_loss), train_acc.append(epoch_acc)

                # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_loss.append(epoch_loss), val_acc.append(epoch_acc)
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    history = [train_loss, train_acc, val_loss, val_acc]

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, history, time_elapsed          # time in seconds




In [10]:
# plot and save the best model
def plot_test_results(hist, title = ""):

    epochs = len(hist[0])

    fig, axes = plt.subplots(1, 2, figsize=(10, 6))

    axes[0].plot(range(1, epochs + 1), hist[0], label='Training')
    axes[0].plot(range(1, epochs + 1), hist[2], label='Validation')
    axes[0].set_xlabel('Epoch')
    axes[0].set_title('Loss')
    axes[0].legend(loc='best')


    axes[1].plot(range(1, epochs + 1), hist[1], label='Training')
    axes[1].plot(range(1, epochs + 1), hist[3], label='Validation')
    axes[1].set_xlabel('Epoch')
    axes[1].set_title('Accuracy')
    axes[1].legend(loc='best')
    
    fig.suptitle(title)

    plt.tight_layout()
    plt.show()


In [11]:
# MN_cnn_fe_models = ["mobile net 2", "mobile net 3 small", "mobile net 3 large", "squeezenet", "resnet50", "vit base 16"]
MN_cnn_fe_models = ["MN_v2_bz_32_ep_15_fe", 
                "MN_v3_s_bz_32_ep_15_fe", 
                "MN_v3_s_bz_64_ep_15_fe", 
                "MN_v3_l_bz_64_ep_25_fe",
                "MN_v3_l_bz_128_ep_15_fe",
                "SQN_bz_128_ep_15_fe",
                "RN50_bz_128_ep_20_fe",
                ]  # `fe` stands for `feature extraction', `ft` stands for `fine-tuning', `lora` stands for `LoRA', `dora` stands for `DoRA'

                # "MN_v3_s_bz_125_ep_15_fe_lora",
                # "MN_v3_s_bz_125_ep_15_fe_dora",
                # "MN_v3_s_bz_125_ep_15_ft",
                # "VIT_bz_32_ep_10_fe"

num_classes = 2
models_save_dir = r'C:\Users\Data_Science\Documents\GitHub\deep-learn-project\models'      # change to your directory
histories_save_dir = r'C:\Users\Data_Science\Documents\GitHub\deep-learn-project\histories'      # change to your directory

if not os.path.exists(models_save_dir):
    os.makedirs(models_save_dir)

if not os.path.exists(histories_save_dir):
    os.makedirs(histories_save_dir)
    
history_titles = ["train_loss", "train_acc", "val_loss", "val_acc"]

In [12]:
def train_and_save_model(model_descriptor):
    print("-------------------------------------------------------------------------------------------------------------------")
    print(f"Model descriptor: {model_descriptor}")
    print("\n")
    # Initialize the model for this run
    model_name = model_descriptor.split("_bz")[0]
    print(f"Model: {model_name}")
    batch_size = int(model_descriptor.split("_bz_")[1].split("_")[0])
    num_epochs = int(model_descriptor.split("_ep_")[1].split("_")[0])
    feature_extract = True if "fe" in model_descriptor else False
    DoRA = True if "dora" in model_descriptor else False
    LoRA = True if "lora" in model_descriptor else False

    model, input_size = initialize_model(model_name, num_classes=2, feature_extract=feature_extract, use_pretrained=True, DoRA=DoRA, LoRA=LoRA)
    print(f"Input size: {input_size}")
    print("\n")
    # Create the dataloaders
    dataloaders, class_names = create_dataloaders(batch_size=batch_size, data_dir=data_dir)
    
    # Send the model to device
    model = model.to(device)
    
    # Gather the parameters to be optimized/updated in this run. If we are
    #  finetuning we will be updating all parameters. However, if we are
    #  doing feature extract method, we will only update the parameters
    #  that we have just initialized, i.e. the parameters with requires_grad
    #  is True.
    optimizer = apply_optimizer(model, feature_extract)
    
    # Setup the loss fn
    criterion = nn.CrossEntropyLoss()

    # Train and evaluate
    model, history, model_time = train_model(model, dataloaders, criterion, optimizer, num_epochs=num_epochs)

    model_save_name = "model_" + model_descriptor + ".pth"
    model_save_path = os.path.join(models_save_dir, model_save_name)
    torch.save(model.state_dict(), model_save_path)
    
    # save the history in csv file with titles: train_loss, train_acc, val_loss, val_acc
    history_save_name = "history_" + model_descriptor + "_" + str(model_time) + ".csv"
    history_save_path = os.path.join(histories_save_dir, history_save_name)
    history_dict = {title: history[i] for i, title in enumerate(history_titles)}
    history_dict_df = pd.DataFrame(history_dict)
    history_dict_df.to_csv(history_save_path, index=False)
    
    print(f"Model saved at: {model_save_path}")
    print("\n")
    print("-------------------------------------------------------------------------------------------------------------------")
    print("\n")
    return model, optimizer, history

### CNN Models with Feature Extraction:

In [13]:
best_cnn_fe_model = None
best_cnn_fe_model_descriptor = None
best_cnn_fe_model_optimizer = None
best_cnn_fe_accuracy = 0.0

# Train the models
for model_descriptor in MN_cnn_fe_models:
    # skip all but the first model
    if model_descriptor != "RN50_bz_128_ep_20_fe":
        continue
    model, optimizer, history = train_and_save_model(model_descriptor)
    
    if best_cnn_fe_model is None:
        best_cnn_fe_model = model
        best_cnn_fe_model_descriptor = model_descriptor
        best_cnn_fe_model_optimizer = optimizer
    else:
        # if validation accuracy is better than the previous best model, update the best model
        if history[3][-1] > best_cnn_fe_accuracy:
            best_cnn_fe_model = model
            best_cnn_fe_model_descriptor = model_descriptor
            best_cnn_fe_model_optimizer = optimizer
            best_cnn_fe_accuracy = history[3][-1]
            best_cnn_fe_history = history

-------------------------------------------------------------------------------------------------------------------
Model descriptor: RN50_bz_128_ep_20_fe


Model: RN50
Input size: 224


Params to learn:
	 fc.weight
	 fc.bias
Epoch 0/19
----------
train Loss: 0.7035 Acc: 0.4613
val Loss: 0.6904 Acc: 0.5774

Epoch 1/19
----------
train Loss: 0.6975 Acc: 0.5119
val Loss: 0.6950 Acc: 0.5536

Epoch 2/19
----------
train Loss: 0.6904 Acc: 0.5357
val Loss: 0.6827 Acc: 0.6131

Epoch 3/19
----------
train Loss: 0.6877 Acc: 0.5506
val Loss: 0.6872 Acc: 0.5714

Epoch 4/19
----------
train Loss: 0.6800 Acc: 0.5729
val Loss: 0.6750 Acc: 0.6012

Epoch 5/19
----------
train Loss: 0.6722 Acc: 0.6012
val Loss: 0.6715 Acc: 0.5952

Epoch 6/19
----------
train Loss: 0.6686 Acc: 0.6176
val Loss: 0.6769 Acc: 0.5595

Epoch 7/19
----------
train Loss: 0.6667 Acc: 0.6176
val Loss: 0.6633 Acc: 0.6071

Epoch 8/19
----------
train Loss: 0.6652 Acc: 0.5997
val Loss: 0.6728 Acc: 0.6012

Epoch 9/19
----------
train

In [28]:
model_name = "resnet50"
feature_extract = True
DoRA = False
LoRA = False
model, input_size = initialize_model(model_name, num_classes=2, feature_extract=feature_extract, use_pretrained=True, DoRA=DoRA, LoRA=LoRA)
print(model)
print(model.fc)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

### Best CNN Feature Extraction with LoRA and DoRA:

In [16]:
best_cnn_fe_model_descriptor_with_lora = best_cnn_fe_model_descriptor + "_lora"
best_cnn_fe_model_descriptor_with_dora = best_cnn_fe_model_descriptor + "_dora"

# Train the best model with LoRA
model_lora, optimizer_lora, history_lora = train_and_save_model(best_cnn_fe_model_descriptor_with_lora)
model_dora, optimizer_dora, history_dora = train_and_save_model(best_cnn_fe_model_descriptor_with_dora)

# Check the best model with LoRA/DoRA
if history_lora[3][-1] > best_cnn_fe_accuracy and history_lora[3][-1] > history_dora[3][-1]:
    # LoRA is better
    best_cnn_fe_model = model_lora
    best_cnn_fe_model_descriptor = best_cnn_fe_model_descriptor_with_lora
    best_cnn_fe_model_optimizer = optimizer_lora
    best_cnn_fe_accuracy = history_lora[3][-1]
    best_cnn_fe_history = history_lora
elif history_dora[3][-1] > best_cnn_fe_accuracy and history_dora[3][-1] > history_lora[3][-1]:
    # DoRA is better
    best_cnn_fe_model = model_dora
    best_cnn_fe_model_descriptor = best_cnn_fe_model_descriptor_with_dora
    best_cnn_fe_model_optimizer = optimizer_dora
    best_cnn_fe_accuracy = history_dora[3][-1]
    best_cnn_fe_history = history_dora

-------------------------------------------------------------------------------------------------------------------
Model descriptor: RN50_bz_128_ep_20_fe_lora


Model: RN50
Input size: 224


Params to learn:
	 fc.weight
	 fc.bias
Epoch 0/19
----------
train Loss: 0.7009 Acc: 0.4881
val Loss: 0.7037 Acc: 0.4821

Epoch 1/19
----------
train Loss: 0.6980 Acc: 0.4866
val Loss: 0.7034 Acc: 0.4702

Epoch 2/19
----------
train Loss: 0.6906 Acc: 0.5491
val Loss: 0.7019 Acc: 0.5060

Epoch 3/19
----------
train Loss: 0.6880 Acc: 0.5491
val Loss: 0.6789 Acc: 0.6131

Epoch 4/19
----------
train Loss: 0.6850 Acc: 0.5402


KeyboardInterrupt: 

### Fine-Tuning on the Best CNN Model:

In [ ]:
best_cnn_fe_model_name = best_cnn_fe_model_descriptor.split("_bz")[0]
print("The best model is: ", best_cnn_fe_model_name)
best_cnn_ft_model_descriptor = best_cnn_fe_model_descriptor.replace("fe", "ft")
print("The best model descriptor is: ", best_cnn_fe_model_descriptor)
print("The best model descriptor with fine-tuning is: ", best_cnn_ft_model_descriptor)
print("The best model accuracy is: ", best_cnn_fe_accuracy)

# train with fine-tuning
model_ft, optimizer_ft, history_ft = train_and_save_model(best_cnn_ft_model_descriptor)
# check if fine-tuning is better
if history_ft[3][-1] > best_cnn_fe_accuracy:
    print("Fine-tuning is better")
    best_cnn_ft_model = model_ft
    best_cnn_ft_model_descriptor = best_cnn_ft_model_descriptor
    best_cnn_ft_model_optimizer = optimizer_ft
    best_cnn_ft_accuracy = history_ft[3][-1]
    best_cnn_ft_history = history_ft
else:
    print("Fine-tuning is not better, keep with your best model")
    print("The best model is: ", best_cnn_fe_model_name)
    

In [ ]:
# # load the best model
# best_cnn_fe_model_name = best_cnn_fe_model_descriptor.split("_bz")[0]
# print("The best model is: ", best_cnn_fe_model_name)
# print("Loading the best model...")
# best_model_save_name = "model_" + best_cnn_fe_model_descriptor + ".pth"
# best_model_save_path = os.path.join(models_save_dir, best_model_save_name)


In [ ]:
# feature_extract = best_cnn_fe_model_descriptor.split("_bz")[0] == "fe"
# DoRA = best_cnn_fe_model_descriptor.split("_bz")[0] == "dora"
# best_cnn_fe_model, _ = initialize_model(best_cnn_fe_model_name, num_classes, feature_extract, use_pretrained=True, DoRA=DoRA)
# best_cnn_fe_model.load_state_dict(torch.load(best_model_save_path))
# best_cnn_fe_model = best_cnn_fe_model.to(device)
# best_cnn_fe_model.eval()

In [ ]:
# # testing function
# def test_model(model, test_loader, criterion, optimizer, num_epochs=25):
#     running_loss = 0.0
#     running_corrects = 0

#     # Iterate over data
#     for inputs, labels in test_loader:
#         inputs = inputs.to(device)
#         labels = labels.to(device)

#         # forward
#         with torch.set_grad_enabled(False):
#             # Get model outputs and calculate loss
#             outputs = model(inputs)
#             loss = criterion(outputs, labels)

#             _, preds = torch.max(outputs, 1)

#         # statistics
#         running_loss += loss.item() * inputs.size(0)
#         running_corrects += torch.sum(preds == labels.data)

#     loss = running_loss / len(test_loader.dataset)
#     acc = running_corrects.double() / len(test_loader.dataset)

#     print('Test Loss: {:.4f} Acc: {:.4f}'.format(loss, acc))

#     return loss, acc

In [ ]:
# # Evaluate on the test set
# print("Test results for the best model", best_cnn_fe_model_descriptor)
# best_model_num_epochs = len(best_cnn_fe_history[0])
# test_loss, test_acc = test_model(best_cnn_fe_model, dataloaders['test'], criterion, best_cnn_fe_model_optimizer, num_epochs=best_model_num_epochs)

# plot_test_results(best_cnn_fe_history, best_cnn_fe_model_descriptor)

In [ ]:
# # loop over all the models and plot the training and validation loss and accuracy
# for model_descriptor in MN_cnn_fe_models:
#     print("-------------------------------------------------------------------------------------------------------------------")
#     print(f"Model descriptor: {model_descriptor}")
#     print("\n")
#     # find a history file name which contains the model descriptor  
#     history_file_name = [f for f in os.listdir(histories_save_dir) if model_descriptor in f][0]
#     if history_file_name is None:
#         print("History file not found")
#         continue
#     model_time = history_file_name.split("_")[-1].split(".")[0]
#     history_save_path = os.path.join(histories_save_dir, history_file_name)
#     df = pd.read_csv(history_save_path)
#     history = [df[title].values for title in history_titles]
#     title = model_descriptor + ", time = " + str(model_time) + "sec"
#     plot_test_results([history["train_loss"], history["train_acc"], history["val_loss"], history["val_acc"]], title)
#     print("-------------------------------------------------------------------------------------------------------------------")